## SQL Case Study Q10-13

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
con = engine.connect()
table_names = engine.table_names()

# Print the table names to the shell
print(table_names)

['Bookings', 'Facilities', 'Members']


C:\Users\bruce\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  This is separate from the ipykernel package so we can avoid doing imports until


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [17]:
query10 = """SELECT sub.facility, sub.revenue 
                 FROM
                     (SELECT f.name AS facility, 
                      SUM(CASE WHEN b.memid = 0
                         THEN b.slots * f.guestcost
                         ELSE b.slots * f.membercost END) AS revenue
                      FROM Bookings as b
                      LEFT JOIN Members AS m 
                         ON m.memid = b.memid
                      LEFT JOIN Facilities as f 
                        ON f.facid = b.facid
                      GROUP BY b.facid)
                  AS sub
                  WHERE sub.revenue < 1000"""

df10 = pd.read_sql(sql = query10, con = con)
df10

,facility,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [11]:
query11 = """
SELECT m1.firstname || ' ' || m1.surname AS name, m2.firstname || ' ' || m2.surname AS recommender
FROM Members AS m1
LEFT JOIN Members AS m2
    ON m1.recommendedby = m2.memid 

"""

df11 = pd.read_sql(sql=query11, con = con)
df11

,name,recommender
0,GUEST GUEST,None
1,Darren Smith,None
2,Tracy Smith,None
3,Tim Rownam,None
4,Janice Joplette,Darren Smith
5,Gerald Butters,Darren Smith
6,Burton Tracy,None
7,Nancy Dare,Janice Joplette
8,Tim Boothe,Tim Rownam
9,Ponder Stibbons,Burton Tracy


Q12: Find the facilities with their usage by member, but not guests

In [6]:
query12 = """
SELECT m.firstname || ' ' || m.surname AS member, f.name AS facility, SUM(b.slots) AS totalSlotsBooked
FROM 
    (SELECT facid, memid, slots FROM Bookings 
     WHERE memid != 0) 
AS b
LEFT JOIN Facilities as f
    ON b.facid = f.facid
LEFT JOIN Members as m
    ON m.memid = b.memid
    
GROUP BY b.memid, facility
"""

df12 = pd.read_sql(sql=query12, con = con)
df12

,member,facility,totalSlotsBooked
0,Darren Smith,Badminton Court,432
1,Darren Smith,Massage Room 1,58
2,Darren Smith,Pool Table,28
3,Darren Smith,Snooker Table,24
4,Darren Smith,Squash Court,30
...,...,...,...
197,John Hunt,Tennis Court 2,12
198,Erica Crumpet,Badminton Court,6
199,Erica Crumpet,Massage Room 1,4
200,Erica Crumpet,Table Tennis,4


Q13: Find the facilities usage by month, but not guests

In [12]:
query13 = """
SELECT f.name AS facility, SUM(b.slots) AS SlotsBookedPerMonth, strftime('%Y-%m', b.starttime) AS Date

FROM 
    (SELECT facid, memid, slots, starttime FROM Bookings 
     WHERE memid != 0) 
AS b
LEFT JOIN Facilities AS f
    ON f.facid = b.facid
GROUP BY facility, Date

"""
df13 = pd.read_sql(sql=query13, con = con)
df13

,facility,SlotsBookedPerMonth,Date
0,Badminton Court,165,2012-07
1,Badminton Court,414,2012-08
2,Badminton Court,507,2012-09
3,Massage Room 1,166,2012-07
4,Massage Room 1,316,2012-08
5,Massage Room 1,402,2012-09
6,Massage Room 2,8,2012-07
7,Massage Room 2,18,2012-08
8,Massage Room 2,28,2012-09
9,Pool Table,110,2012-07
